In [ ]:
!pip install git+https://github.com/Healthcare-Robotics/assistive-gym.git

CPU times: user 362 ms, sys: 174 ms, total: 536 ms
Wall time: 46.9 s


In [ ]:
!pip install stable-baselines3

In [ ]:
import assistive_gym
from stable_baselines3 import PPO
from stable_baselines3.common.cmd_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common import evaluation
import os, multiprocessing
from numpngw import write_apng
from IPython.display import Image

In [ ]:
env_name = 'FeedingSawyer-v1'
num_processes = multiprocessing.cpu_count()
env = make_vec_env(env_name, n_envs=num_processes)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=100000)
model.save(os.path.join('baselines3_ppo', env_name))
env.save(os.path.join('baselines3_ppo', '%s_vec_normalize.pkl' % env_name))

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -155     |
| time/              |          |
|    fps             | 48       |
|    iterations      | 1        |
|    time_elapsed    | 84       |
|    total_timesteps | 4096     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -152        |
| time/                   |             |
|    fps                  | 48          |
|    iterations           | 2           |
|    time_elapsed         | 168         |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.011992622 |
|    clip_fraction        | 0.136       |
|    clip_range           | 0.2         |
|    entropy_loss         | -9.92       |
|    explained_variance   | 0.341       |
|    learning

In [ ]:
model = PPO.load(os.path.join('baselines3_ppo', env_name))
eval_env = make_vec_env(env_name, n_envs=1)
eval_env = VecNormalize.load(os.path.join('baselines3_ppo', '%s_vec_normalize.pkl' % env_name), eval_env)
eval_env.training = False
eval_env.norm_reward = False
eval_env.envs[0].setup_camera(camera_eye=[0.5, -0.75, 1.5], camera_target=[-0.2, 0, 0.75], fov=60, camera_width=1920, camera_height=1080)

obs = eval_env.reset()
frames = []
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = eval_env.step(action)
    img, depth = eval_env.envs[0].get_camera_image_depth()
    frames.append(img)
write_apng('output_baselines3_%s.png' % env_name, frames, delay=100)

In [ ]:
Image(filename='output_baselines3_%s.png' % env_name)

In [ ]:
mean_reward, std_reward = evaluation.evaluate_policy(model, eval_env, n_eval_episodes=200, deterministic=True)
print(f'mean_reward={mean_reward:.2f} +/- {std_reward}')

mean_reward=-49.43 +/- 76.27664821810389


In [ ]:
import os, sys
from google.colab import drive
import cv2
from google.colab.patches import cv2_imshow
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/Colab Notebooks/OPENCV")
sys.path.append("/content/drive/My Drive/Colab Notebooks/OPENCV")

import cv2
import numpy as np
frame_width = 1920
frame_height = 1080
   
size = (frame_width, frame_height)

result = cv2.VideoWriter('out.avi', cv2.VideoWriter_fourcc(*'XVID'), 1, size)
print(len(frames))
i = 0
for frame in frames:
  i+=1
  if i == 150:
    cv2.imwrite("PPO_mid.jpg", frame)
    print(frame)
  result.write(frame)
result.release()
cv2.destroyAllWindows()

Mounted at /content/drive
200
[[[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 [[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 [[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 ...

 [[195 175 142 255]
  [194 176 142 255]
  [194 176 142 255]
  ...
  [ 59 110 153 255]
  [ 59 110 153 255]
  [ 59 110 153 255]]

 [[195 175 142 255]
  [195 175 142 255]
  [195 175 142 255]
  ...
  [ 59 110 153 255]
  [ 59 110 153 255]
  [ 59 110 153 255]]

 [[196 175 143 255]
  [196 175 143 255]
  [193 172 142 255]
  ...
  [ 59 110 153 255]
  [ 59 110 153 255]
  [ 59 110 153 255]]]


In [ ]:
from stable_baselines3 import SAC

env_name = 'FeedingSawyer-v1'
num_processes = multiprocessing.cpu_count()
env = make_vec_env(env_name, n_envs=num_processes)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)
model = SAC('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=100000)
model.save(os.path.join('baselines3_sac', env_name))
env.save(os.path.join('baselines3_sac', '%s_vec_normalize.pkl' % env_name))

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -122     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 28       |
|    time_elapsed    | 28       |
|    total_timesteps | 800      |
| train/             |          |
|    actor_loss      | -13.6    |
|    critic_loss     | 1        |
|    ent_coef        | 0.901    |
|    ent_coef_loss   | -1.23    |
|    learning_rate   | 0.0003   |
|    n_updates       | 349      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -140     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 29       |
|    time_elapsed    | 54       |
|    total_timesteps | 1600     |
| train/             |          |
|    actor_loss      | -18.6    |
|    critic_loss     | 0.17    

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/save_util.py:276: UserWarning: Path 'baselines3_sac' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [ ]:
model = SAC.load(os.path.join('baselines3_sac', env_name))
eval_env = make_vec_env(env_name, n_envs=1)
eval_env = VecNormalize.load(os.path.join('baselines3_sac', '%s_vec_normalize.pkl' % env_name), eval_env)
eval_env.training = False
eval_env.norm_reward = False
eval_env.envs[0].setup_camera(camera_eye=[0.5, -0.75, 1.5], camera_target=[-0.2, 0, 0.75], fov=60, camera_width=1920, camera_height=1080)

obs = eval_env.reset()
frames = []
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = eval_env.step(action)
    img, depth = eval_env.envs[0].get_camera_image_depth()
    frames.append(img)
write_apng('output_baselines3_sac_%s.png' % env_name, frames, delay=100)

In [ ]:
Image(filename='output_baselines3_sac_%s.png' % env_name)

In [ ]:
mean_reward, std_reward = evaluation.evaluate_policy(model, eval_env, n_eval_episodes=200, deterministic=True)
print(f'mean_reward={mean_reward:.2f} +/- {std_reward}')

mean_reward=-215.62 +/- 54.09327023746733


In [ ]:
frame_width = 1920
frame_height = 1080
   
size = (frame_width, frame_height)

result = cv2.VideoWriter('out_sac.avi', cv2.VideoWriter_fourcc(*'XVID'), 1, size)
print(len(frames))
i = 0
for frame in frames:
  i+=1
  if i == 150:
    cv2.imwrite("SAC_mid.jpg", frame)
    print(frame)
  result.write(frame)
result.release()
cv2.destroyAllWindows()

200
[[[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 [[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 [[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 ...

 [[195 175 142 255]
  [194 176 142 255]
  [194 176 142 255]
  ...
  [ 59 110 153 255]
  [ 59 110 153 255]
  [ 59 110 153 255]]

 [[195 175 142 255]
  [195 175 142 255]
  [195 175 142 255]
  ...
  [ 59 110 153 255]
  [ 59 110 153 255]
  [ 59 110 153 255]]

 [[196 175 143 255]
  [196 175 143 255]
  [193 172 142 255]
  ...
  [ 59 110 153 255]
  [ 59 110 153 255]
  [ 59 110 153 255]]]
